In [1]:
import numpy as np
def pooling(image, pooling_size, W_O, H_O):
    index = np.zeros((W_O, H_O))
    output = np.zeros((W_O, H_O))
    for y in range(0, image.shape[1], pooling_size):
        for x in range(0, image.shape[0], pooling_size):
            index[x//2, y//2] = np.argmax(image[x:x+pooling_size, y:y+pooling_size])
            output[x//2, y//2] = (image[x:x+pooling_size, y:y+pooling_size]).max()
    return output, index


# def pooling(image, pooling_size, W_O, H_O, I):

#     output = np.zeros((W_O, H_O))
#     for y in range(0, image.shape[1], pooling_size):
#         for x in range(0, image.shape[0], pooling_size):
#             output[x//2, y//2] = (image[x:x+pooling_size,
#                                   y:y+pooling_size]).mean()
#     return output


def poolingLayer(input_feature_map, pooling_size=2):   
    # Check whether it's 3D or not.
    W_O = input_feature_map.shape[1]//2
    H_O = input_feature_map.shape[2]//2
    C_I = input_feature_map.shape[0]
    output_feature_map = np.zeros((C_I, W_O, H_O))
    for cin in range(C_I):
        output_feature_map[cin] = pooling(input_feature_map[cin], pooling_size, W_O, H_O)
    return output_feature_map


# A = np.random.randint(0, 16, (3, 4, 4))
# poolingLayer(A, 3, 2)


In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
#%matplotlib inline, already solved by VS code
np.set_printoptions(precision = 2)
class MLP(object):
    '''Multi_Layer Perceptron, Fully-Connected Layer'''
    
    def __init__(self, Layers = (2, 2, 3), BatchSize = 4):
        self.bs = BatchSize
        self.lr = float()
        self.LeakyRate = float()
        self.act = str()
        self.net = [dict() for _ in range(len(Layers))]# Every element in the list is a dictionary 

        self.net[0]['a'] = np.zeros((Layers[0], self.bs), dtype = 'float16')
        self.net[0]['dJda'] = np.zeros(self.net[0]['a'].shape, dtype = 'float16')
        
        for i in range(1, len(Layers)):
            self.net[i]['a'] = np.zeros((Layers[i], self.bs), dtype = 'float16')
            self.net[i]['z'] = np.zeros(self.net[i]['a'].shape, dtype = 'float16')
            self.net[i]['W'] = np.random.randn(Layers[i], Layers[i - 1]).astype('float16')
            self.net[i]['b'] = np.random.randn(Layers[i], 1).astype('float16')
            self.net[i]['dJda'] = np.zeros(self.net[i]['a'].shape, dtype = 'float16')
            self.net[i]['dJdz'] = np.zeros(self.net[i]['z'].shape, dtype = 'float16')
            self.net[i]['dJdW'] = np.zeros(self.net[i]['W'].shape, dtype = 'float16')
            self.net[i]['dJdb'] = np.zeros(self.net[i]['b'].shape, dtype = 'float16')
        
        self.p = np.zeros(self.net[-1]['a'].shape, dtype = 'float16') # Softmax Out
        self.dJdp = np.zeros(self.p.shape, dtype='float16')

        self.yhat = np.zeros(self.bs, dtype=int) # Predicted Answer
        self.yhat_onehot = np.zeros(self.p.shape, dtype=int)
        self.y_onehot = np.zeros(self.p.shape, dtype=int)

        self.J = []
        self.W_trace = []


    def softmax(self, a):
        delta = 1.0
        return np.exp(a + delta) / np.sum(np.exp(a + delta), axis=0)
    def sigmoid(self, z):
        return 1.0 / (1.0 + np.exp(-1.0 * z))
    def sigmoidPrime(self, a): # sigmoid's derivative
        return a * (1.0 -a)
    def ReLU(self, z):
        a = np.copy(z)
        a[a < 0] = 0.0
        return a
    def ReLUPrime(self, a):
        dadz = np.copy(a)
        dadz[a > 0] = 1.0
        return dadz
    def activation(self, z):
        a = np.copy(z)
        if self.act == "sigmoid":
            a = self.sigmoid(z)
        #elif self.act == "ReLU":
        #    a = self.LeakyReLU
        else:
            print("Activation Selection Error")
        return a
    def activationPrime(self, a):
        if self.act == "sigmoid":
            z = self.sigmoidPrime(a)
        else:
            print("Activation Selection Error")
        return z
    # Warren
    def forward(self, x): # "copyto" looks like directly manipulate the value at some address
        np.copyto(self.net[0]['a'], x) # Copy input into a of Layer 0

        for i in range(1, len(self.net)): # Start from 0; Let i start from 1
            np.copyto(self.net[i]['z'], np.dot(self.net[i]['W'], self.net[i-1]['a']) + self.net[i]['b']) # iterator
            np.copyto(self.net[i]['a'], self.activation(self.net[i]['z']))
        
        np.copyto(self.p, self.softmax(self.net[-1]['a']))
        np.copyto(self.yhat, np.argmax(self.p, axis=0))# Final predicated answer
        return

    def loss(self, y):
        '''only a "1" '''
        self.y_onehot.fill(0)
        for i in range(self.bs):
            self.y_onehot[y[i], i] = 1 # Compare & Turn on
        loss_value = -1.0 * np.sum(self.y_onehot * np.log(self.p)) / self.bs

        self.J.append(loss_value) # Look 
# fffff
        W = []
        for i in range(1, len(self.net)):
            W.extend(np.ravel(self.net[i]['W'])) # ravel =? extend
            W.extend(np.ravel(self.net[i]['b']))
        self.W_trace.append(W)
        return

    def backprop(self):
        self.dJdp = 1.0 / (1.0 - self.y_onehot - self.p)

        dpda = np.array([[self.p[i, :] * (1.0 - self.p[j, :]) if i == j
                          else -1 * self.p[i, :] * self.p[j, :]
                          for i in range(self.p.shape[0])]
                          for j in range(self.p.shape[0])])
        for i in range(self.bs):
            self.net[-1]['dJda'][:, i] = np.dot(dpda[:, :, i], self.dJdp[:, i])

        for i in range((len(self.net) - 1), 0, -1):# computed function 
            np.copyto(self.net[i]['dJdz'], (self.net[i]['dJda'] * self.activationPrime(self.net[i]['a'])))

            np.copyto(self.net[i]['dJdb'], np.mean(self.net[i]['dJdz'], axis = 1)[:, None])

            np.copyto(self.net[i]['dJdW'], np.dot(self.net[i]['dJdz'], self.net[i-1]['a'].T) / self.bs)

            np.copyto(self.net[i - 1]['dJda'], np.dot((self.net[i]['W']).T, self.net[i]['dJdz']))
        return

    def update(self):
        for i in range(1, len(self.net)):
            np.copyto(self.net[i]['W'], self.net[i]['W'] - self.lr * self.net[i]['dJdW']) # += may work
            #self.net[i]['W'] -= self.lr * self.net[i]['dJdW']
            np.copyto(self.net[i]['b'], self.net[i]['b'] - self.lr * self.net[i]['dJdb'])
        return
    
    def train(self, train_x, train_y, epoch_count = 1000, lr = 0.01, act = "sigmoid", LeakyRate = 0.1):
        self.lr = lr
        self.act = act
        self.LeakyRate = LeakyRate

        for _ in range(epoch_count): # constant times for, solved:how to eliminate warning the variable isn't used 
            for i in range(train_x.shape[1] // self.bs): 
                x = train_x[:, i * self.bs : (i + 1) * self.bs]
                y = train_y[i * self.bs : (i + 1) * self.bs]
                self.forward(x)
                self.loss(y)
                self.backprop()
                self.update()
        return

    def inference(self, inference_x):
        yhat = []
        for i in range(inference_x.shape[1]//self.bs):
            x = inference_x[:, i * self.bs:(i + 1) * self.bs]
            self.forward(x)
            yhat.extend(list(self.yhat))
        return yhat
    
    def plot_W(self):
        curve = [[] for i in range(len(self.W_trace[0]))]
        for j in range(len(self.W_trace[0])):
            for i in range(len(self.W_trace)):
                curve[j].append(self.W_trace[i][j])
        for c in curve:
                plt.plot(c)

        return


In [3]:
z, indecies = pooling(A[0], 2, 2, 2)

NameError: name 'A' is not defined